<a href="https://colab.research.google.com/github/DolgorTs/resumeandprojects.github/blob/main/%22%D0%9F%D0%BE%D0%B8%D1%81%D0%BA_%D0%B0%D0%BD%D0%BE%D0%BC%D0%B0%D0%BB%D0%B8%D0%B9_%D0%B2_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B8_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D0%BF%D1%80%D0%BE%D0%BF%D1%83%D1%89%D0%B5%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8_%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F%D0%BC%D0%B8_(%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82_Horses)_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В рамках данного исследования в датасете horses, содержащего информацию о больных лошадях, проведено следующее:

1. Датасет загружен через файл csv
2. Определены типы переменных
3. Расчитаны базовые статистики
4. Определены и проработаны выбросы
5. Определены и проработаны пропущенные значения.


In [ ]:
# подгрузим библиотеки
import pandas as pd
import numpy as np

In [ ]:
file_ = open('horse_data.names')
print(file_.read())

1. TItle: Horse Colic database

2. Source Information
   -- Creators: Mary McLeish & Matt Cecile
	  	Department of Computer Science
		University of Guelph
		Guelph, Ontario, Canada N1G 2W1
		mdmcleish@water.waterloo.edu
   -- Donor:    Will Taylor (taylor@pluto.arc.nasa.gov)
   -- Date:     8/6/89

3. Past Usage:
   -- Unknown

4. Relevant Information:

   -- 2 data files
      -- horse-colic.data: 300 training instances
      -- horse-colic.test: 68 test instances
   -- Possible class attributes: 24 (whether lesion is surgical)
     -- others include: 23, 25, 26, and 27
   -- Many Data types: (continuous, discrete, and nominal)

5. Number of Instances: 368 (300 for training, 68 for testing)

6. Number of attributes: 28

7. Attribute Information:

  1:  surgery?
          1 = Yes, it had surgery
          2 = It was treated without surgery

  2:  Age
          1 = Adult horse
          2 = Young (< 6 months)

  3:  Hospital Number
          - numeric id
          - the case number assi

In [ ]:
# Загрузка данных

column_names = ['surgery', 'age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']

df = pd.read_csv('horse_data.csv', usecols=[0, 1, 3, 4, 5, 6, 10, 22], names = column_names, na_values = '?')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     299 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


**Количественные (непрерывные):**
- rectal temperature
- pulse
- respiratory rate

Для них можно применить метод describe(), найти базовые статистики, применить межквартильный размах для поиска выбросов.














In [ ]:
df['rectal temperature'].describe()

count    240.000000
mean      38.167917
std        0.732289
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64

In [ ]:
print('mean - среднее ариф.')
print(df['rectal temperature'].mean())
print('-----------')
print('median:')
print(df['rectal temperature'].median())
print('-----------')
print('mode:')
print(df['rectal temperature'].mode()[0])
print('-----------')
print('std - стандарт. отклонение')
print(df['rectal temperature'].std())
print('-----------')
print('var - дисперсия:')  #Чем выше дисперсия, тем выше разброс значений в ряде данных.
print(df['rectal temperature'].var())

mean - среднее ариф.
38.16791666666667
-----------
median:
38.2
-----------
mode:
38.0
-----------
std - стандарт. отклонение
0.7322886641121578
-----------
var - дисперсия:
0.5362466875871686


На основе базовых статистик можно сделать вывод, что в среднем ректальная температура у лошадей, испытывающих кишечные колики, около 38, что незначительно превышает нормальную температуру 37.8. Первый квартиль равен значению нормальной температуры, значит, что 25% лошадей переживают поздний шок, а 75 % лошадей испытывают инфекцию.
Также видно, что данные распределены нормально, т.к. разница между средней и медианой небольшая около 0,8. Разброс значений не особо существенный, т.к. величина дисперсии и разница между максимальным и минимальным значениями небольшая.  



In [ ]:
q1_rtemp = df['rectal temperature'].quantile(0.25)
q3_rtemp = df['rectal temperature'].quantile(0.75)
iqr_rtemp = q3_rtemp - q1_rtemp
lower_bound_rtemp = q1_rtemp - (1.5 * iqr_rtemp)
upper_bound_rtemp = q3_rtemp + (1.5 * iqr_rtemp)
print('первый квартиль - ',q1_rtemp)
print('третий квартиль - ',q3_rtemp)
print('межквартильный размах - ',iqr_rtemp)
print('нижняя граница выбросов -', lower_bound_rtemp)
print('верхняя граница выбросов - ',upper_bound_rtemp)

# remove_outliers_rtemp = df[df['rectal temperature'].between(lower_bound_rtemp, upper_bound_rtemp, inclusive=True)] #новый датафрейм без выбросов
# remove_outliers_rtemp

первый квартиль -  37.8
третий квартиль -  38.5
межквартильный размах -  0.7000000000000028
нижняя граница выбросов - 36.74999999999999
верхняя граница выбросов -  39.550000000000004


Нижняя и верхняя границы выбросов, по-моему мнению, имеют вполне правдоподобные значения. Т.к. мы исследуем температуру больных лошадей, т.е. животных, и она вполне естественно может колебаться в значениях границ выбросов. Поэтому можно обойтись без удаления значений выбросов.

In [ ]:
df['pulse'].describe()

count    276.000000
mean      71.913043
std       28.630557
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

In [ ]:
print('mean - среднее ариф.')
print(df['pulse'].mean())
print('-----------')
print('median:')
print(df['pulse'].median())
print('-----------')
print('mode:')
print(df['pulse'].mode()[0])
print('-----------')
print('std - стандарт. отклонение')
print(df['pulse'].std())
print('-----------')
print('var - дисперсия:')  #Чем выше дисперсия, тем выше разброс значений в ряде данных.
print(df['pulse'].var())

mean - среднее ариф.
71.91304347826087
-----------
median:
64.0
-----------
mode:
48.0
-----------
std - стандарт. отклонение
28.630556660735
-----------
var - дисперсия:
819.7087747035573


Пульс 30-40 ударов в минуту является нормальным показателем для взрослой лошади. В наших данных первый квартиль выше нормального показателя, значит, что меньше 25% лошадей имеют здоровый пульс. При чем у спортивных лошадей пульс должен быть около 20-25, мы получили минимальное значение пульса 30, следует вывод, что спортивных лошадей в нашей выборке нет. Также выходит, что 75% лошадей страдают сердечно-сосудистыми заболеваними и болезнями, т.к. их пульс выше нормы.
Данные распределены нормально, но имеется большой разброс значений, во-первых, разница между мин и макс - 154, и во-вторых, дисперсия имеет огромную величину.

In [ ]:
q1_pulse = df['pulse'].quantile(0.25)
q3_pulse = df['pulse'].quantile(0.75)
iqr_pulse = q3_pulse - q1_pulse
lower_bound_pulse = q1_pulse - (1.5 * iqr_pulse)
upper_bound_pulse = q3_pulse + (1.5 * iqr_pulse)
print('первый квартиль - ',q1_pulse)
print('третий квартиль - ',q3_pulse)
print('межквартильный размах - ',iqr_pulse)
print('нижняя граница выбросов -', lower_bound_pulse)
print('верхняя граница выбросов - ',upper_bound_pulse)
remove_outliers_pulse = df[df['pulse'].between(lower_bound_pulse, upper_bound_pulse, inclusive=True)] #новый датафрейм без выбросов
remove_outliers_pulse

первый квартиль -  48.0
третий квартиль -  88.0
межквартильный размах -  40.0
нижняя граница выбросов - -12.0
верхняя граница выбросов -  148.0


<ipython-input-111-1e1bde374ab2>:11: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  remove_outliers_pulse = df[df['pulse'].between(lower_bound_pulse, upper_bound_pulse, inclusive=True)] #новый датафрейм без выбросов


,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
6,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


В данном случае, можно удалить выбросы. Нижняя граница выбросов имеет отрицательное значение из-за высокого разброса данных, хотя минимальный пульс равен 30. При чем вырхняя граница выбросов - 148 значительно превышает норму, несмотря на болезненость исследуемых лошадей. Возможно, данные введены неверно.

In [ ]:
df['respiratory rate'].describe()

count    242.000000
mean      30.417355
std       17.642231
min        8.000000
25%       18.500000
50%       24.500000
75%       36.000000
max       96.000000
Name: respiratory rate, dtype: float64

In [ ]:
print('mean - среднее ариф.')
print(df['respiratory rate'].mean())
print('-----------')
print('median:')
print(df['respiratory rate'].median())
print('-----------')
print('mode:')
print(df['respiratory rate'].mode()[0])
print('-----------')
print('std - стандарт. отклонение')
print(df['respiratory rate'].std())
print('-----------')
print('var - дисперсия:')  #Чем выше дисперсия, тем выше разброс значений в ряде данных.
print(df['respiratory rate'].var())

mean - среднее ариф.
30.417355371900825
-----------
median:
24.5
-----------
mode:
20.0
-----------
std - стандарт. отклонение
17.64223138513467
-----------
var - дисперсия:
311.2483282466308


Нормальная частота дыхания 8 - 10. Средние значения существенно превышают нормальные значения, что говорит о преобладающей болезнености лошадей. Распределение нормальное. Разброс данных очень большой, тк. дисперсия больше 300.

In [ ]:
q1_resp = df['respiratory rate'].quantile(0.25)
q3_resp = df['respiratory rate'].quantile(0.75)
iqr_resp = q3_resp - q1_resp
lower_bound_resp = q1_resp - (1.5 * iqr_resp)
upper_bound_resp = q3_resp + (1.5 * iqr_resp)
print('первый квартиль - ',q1_resp)
print('третий квартиль - ',q3_resp)
print('межквартильный размах - ',iqr_resp)
print('нижняя граница выбросов -', lower_bound_resp)
print('верхняя граница выбросов - ',upper_bound_resp)

remove_outliers_resp = df[df['respiratory rate'].between(lower_bound_resp, upper_bound_resp, inclusive=True)] #новый датафрейм без выбросов
remove_outliers_resp

первый квартиль -  18.5
третий квартиль -  36.0
межквартильный размах -  17.5
нижняя граница выбросов - -7.75
верхняя граница выбросов -  62.25


<ipython-input-114-f317b9ad22e4>:12: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  remove_outliers_resp = df[df['respiratory rate'].between(lower_bound_resp, upper_bound_resp, inclusive=True)] #новый датафрейм без выбросов


,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
6,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...
294,2.0,1,38.5,40.0,16.0,1.0,2.0,1.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


Разумней убрать выбросы по нижней и верхней границе. Нижняя граница отрицательная из-за высокого разброса данных,и даже меньше минимального значения. Вырхняя граница намного выше средних значений. Скорей всего данные внесены некорректно.


**Категориальные:**

*   surgery?
*   Age
*   temperature of extremities
*   pain
*   outcome

Для них можно найти моды, проанализировать распределение по категориям, проверить уникальные значения категорий на предмет ошибок и соответствия описанию данных. Помогут методы mode(), value_counts(), unique() .

In [ ]:
print('mode:')
print(df['surgery'].mode()[0])
print('--------')
print('распределение:')
print(df['surgery'].value_counts())
print('--------')
print('уникальные значения:')
print(df['surgery'].unique())

mode:
1.0
--------
распределение:
1.0    180
2.0    119
Name: surgery, dtype: int64
--------
уникальные значения:
[ 2.  1. nan]


60 % лошадей прооперированы. В столбце всего лишь одно пропущенное значение (300 - (180+119)), что говорит о полноте данных.

In [ ]:
print('mode:')
print(df['age'].mode()[0])
print('--------')
print('распределение:')
print(df['age'].value_counts())
print('--------')
print('уникальные значения:')
print(df['age'].unique())

mode:
1
--------
распределение:
1    276
9     24
Name: age, dtype: int64
--------
уникальные значения:
[1 9]


92 % лошадей больше 6 месяцев. В столбце нет пропущеных значений. Но в датафрейм не соответсвует описанию, т.к. есть значения только 1 и 9, а значения 9 в описании нет.

In [ ]:
print('mode:')
print(df['temperature of extremities'].mode()[0])
print('--------')
print('распределение:')
print(df['temperature of extremities'].value_counts())
print('--------')
print('уникальные значения:')
print(df['temperature of extremities'].unique())

mode:
3.0
--------
распределение:
3.0    109
1.0     78
2.0     30
4.0     27
Name: temperature of extremities, dtype: int64
--------
уникальные значения:
[ 3. nan  1.  4.  2.]


Заполнено 244 значения. Из них:
-  44,6 % - лошади с холодной температурой конечностей, что подтверждает и мода
-  31,9 % - лошади с нормальной температурой конечностей
-  12,3 % - лошади с теплой температурой конечностей
-  11,1 % - лошади с очень холодной температурой конечностей

55,7% лошадей испытывают шок, т.к. имеют холодную их температуру конечностей.
Возможно у 31,9 % лошадей также высокая ректальная температура, т.к. она коррелирует с высокой температурой конечностей.

Данные соответсвую описанию датафрейма.
В столбце присутвуют пропущенные значения - 56 строк (18,6%), что говорит, о средней полноте данных.




In [ ]:
print('mode:')
print(df['pain'].mode()[0])
print('--------')
print('распределение:')
print(df['pain'].value_counts())
print('--------')
print('уникальные значения:')
print(df['pain'].unique())

mode:
3.0
--------
распределение:
3.0    67
2.0    59
5.0    42
4.0    39
1.0    38
Name: pain, dtype: int64
--------
уникальные значения:
[ 5.  3.  2. nan  4.  1.]


Заполнено 245 значений:
- 27,3 % лошадей испытывают периодическую легкую боль
- 24 % - в депресии
- 17,2 % - постоянную сильную боль
- 15,9 % - периодическую сильную боль
- 15,5 % - тревожность без боли

Как видно, данные распределены вполне равномерны, сильных разлий нет, к примеру, 80 % - лошадей имеют сильную боль, 20% - периодическую.

Возможно 15,5% лошадей потребуется операция.

Данные соответсвую описанию датафрейма.
В столбце присутвуют пропущенные значения - 55 строк (18,3%), что говорит, о средней полноте данных.






In [ ]:
print('mode:')
print(df['outcome'].mode()[0])
print('--------')
print('распределение:')
print(df['outcome'].value_counts())
print('--------')
print('уникальные значения:')
print(df['outcome'].unique())

mode:
1.0
--------
распределение:
1.0    178
2.0     77
3.0     44
Name: outcome, dtype: int64
--------
уникальные значения:
[ 2.  3.  1. nan]


Заполнено 299 значения. Пропущено только 1. Данные вполне полные.

59,5 % лошадей - живы
25,7 % - умерли
14,7 % - подвергнуты эвтаназии

Чуть больше половины лошадей живы.




In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     299 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


**ПРОПУСКИ**

**категориальные**
- surgery?,
- Age,
- temperature of extremities,
- pain,
- outcome

замена модой

**Количественные (непрерывные):**
- rectal temperature
- pulse
- respiratory rate


>
>



- !Замена медианой, если есть выбросы


- !Замена средней, если выбросов - нет

В датафрейме пропущено около 30% значений, довольно большая часть, поэтому удалять пропущенные значения лучше не стоит.
Поэтому заменим пропуски на основе других признаков, применяя группировку.
Для категориальных переменных - через моду и группировку. Для количественных через медиану и среднюю.




In [ ]:
df.isnull().sum()

surgery                        1
age                            0
rectal temperature            60
pulse                         24
respiratory rate              58
temperature of extremities    56
pain                          55
outcome                        1
dtype: int64

In [ ]:
df_new = df.copy()

In [ ]:
# заменим модой пропуски в столбце surgery через группировку по возрасту. То есть подвергалась ли лошадь операции зависит от возраста.

print(df['surgery'].value_counts())

df_new['surgery'] = df['surgery'].fillna(df.groupby('age')['surgery'].transform(lambda x: x.mode()[0]))
df_new.info()

1.0    180
2.0    119
Name: surgery, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     300 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  300 non-null    float64
 6   pain                        265 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [ ]:
# добавление пропущеного значение не особо повлияло на статистику данных, т.к. заменено только 1 значение.
print('mode:')
print(df['surgery'].mode()[0])
print('--------')
print('распределение:')
print(df['surgery'].value_counts())
print('--------')
print('уникальные значения:')
print(df['surgery'].unique())
print('------------------------')
print('mode:')
print(df_new['surgery'].mode()[0])
print('--------')
print('распределение:')
print(df_new['surgery'].value_counts())
print('--------')
print('уникальные значения:')
print(df_new['surgery'].unique())

mode:
1.0
--------
распределение:
1.0    180
2.0    119
Name: surgery, dtype: int64
--------
уникальные значения:
[ 2.  1. nan]
------------------------
mode:
1.0
--------
распределение:
1.0    181
2.0    119
Name: surgery, dtype: int64
--------
уникальные значения:
[2. 1.]


In [ ]:
# температура конечностей зависит от операции

print(df['temperature of extremities'].value_counts())

df_new['temperature of extremities'] = df['temperature of extremities'].fillna(df.groupby('surgery')['temperature of extremities'].transform(lambda x: x.mode()[0]))
df_new.info()

3.0    109
1.0     78
2.0     30
4.0     27
Name: temperature of extremities, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     300 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  300 non-null    float64
 6   pain                        300 non-null    float64
 7   outcome                     300 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [ ]:
# после замены пропусков изменилось распределение в сторону моды.
print('mode:')
print(df['temperature of extremities'].mode()[0])
print('--------')
print('распределение:')
print(df['temperature of extremities'].value_counts())
print('--------')
print('уникальные значения:')
print(df['temperature of extremities'].unique())
print('------------------------')
print('mode:')
print(df_new['temperature of extremities'].mode()[0])
print('--------')
print('распределение:')
print(df_new['temperature of extremities'].value_counts())
print('--------')
print('уникальные значения:')
print(df_new['temperature of extremities'].unique())

mode:
3.0
--------
распределение:
3.0    109
1.0     78
2.0     30
4.0     27
Name: temperature of extremities, dtype: int64
--------
уникальные значения:
[ 3. nan  1.  4.  2.]
------------------------
mode:
3.0
--------
распределение:
3.0    165
1.0     78
2.0     30
4.0     27
Name: temperature of extremities, dtype: int64
--------
уникальные значения:
[3. 1. 4. 2.]


In [ ]:
# уровень боли зависит от операции

print(df['pain'].value_counts())

df_new['pain'] = df['pain'].fillna(df.groupby('surgery')['pain'].transform(lambda x: x.mode()[0]))
df_new.info()

3.0    67
2.0    59
5.0    42
4.0    39
1.0    38
Name: pain, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     300 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  300 non-null    float64
 6   pain                        300 non-null    float64
 7   outcome                     300 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [ ]:
# после замены пропусков изменилось распределение в сторону моды.
print('mode:')
print(df['pain'].mode()[0])
print('--------')
print('распределение:')
print(df['pain'].value_counts())
print('--------')
print('уникальные значения:')
print(df['pain'].unique())
print('------------------------')
print('mode:')
print(df_new['pain'].mode()[0])
print('--------')
print('распределение:')
print(df_new['pain'].value_counts())
print('--------')
print('уникальные значения:')
print(df_new['pain'].unique())

mode:
3.0
--------
распределение:
3.0    67
2.0    59
5.0    42
4.0    39
1.0    38
Name: pain, dtype: int64
--------
уникальные значения:
[ 5.  3.  2. nan  4.  1.]
------------------------
mode:
3.0
--------
распределение:
3.0    104
2.0     59
1.0     56
5.0     42
4.0     39
Name: pain, dtype: int64
--------
уникальные значения:
[5. 3. 2. 1. 4.]


In [ ]:
# заменим пропуски в столбце outcome через моду по группировке age

print(df['outcome'].value_counts())

df_new['outcome'] = df['outcome'].fillna(df.groupby('age')['outcome'].transform(lambda x: x.mode()[0]))
df_new.info()

1.0    178
2.0     77
3.0     44
Name: outcome, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     300 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          300 non-null    float64
 3   pulse                       300 non-null    float64
 4   respiratory rate            300 non-null    float64
 5   temperature of extremities  300 non-null    float64
 6   pain                        300 non-null    float64
 7   outcome                     300 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [ ]:
# после замены пропусков изменилось распределение в сторону моды.
print('mode:')
print(df['outcome'].mode()[0])
print('--------')
print('распределение:')
print(df['outcome'].value_counts())
print('--------')
print('уникальные значения:')
print(df['outcome'].unique())
print('------------------------')
print('mode:')
print(df_new['outcome'].mode()[0])
print('--------')
print('распределение:')
print(df_new['outcome'].value_counts())
print('--------')
print('уникальные значения:')
print(df_new['outcome'].unique())

mode:
1.0
--------
распределение:
1.0    178
2.0     77
3.0     44
Name: outcome, dtype: int64
--------
уникальные значения:
[ 2.  3.  1. nan]
------------------------
mode:
1.0
--------
распределение:
1.0    179
2.0     77
3.0     44
Name: outcome, dtype: int64
--------
уникальные значения:
[2. 3. 1.]


In [ ]:
# Так как существенных выбросов при анализе в столбце rectal temperature не было, то заменим пропуски среднем

df_new['rectal temperature'] = df['rectal temperature'].fillna(df['rectal temperature'].mean())
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     300 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          300 non-null    float64
 3   pulse                       300 non-null    float64
 4   respiratory rate            300 non-null    float64
 5   temperature of extremities  300 non-null    float64
 6   pain                        300 non-null    float64
 7   outcome                     300 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [ ]:
# Как видно из сравнения статистки после замены пропусков данные изменились несильно.
print(df['rectal temperature'].describe())
print('-----------------------------------')
print(df_new['rectal temperature'].describe())

count    240.000000
mean      38.167917
std        0.732289
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64
-----------------------------------
count    300.000000
mean      38.167917
std        0.654705
min       35.400000
25%       37.900000
50%       38.167917
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64


In [ ]:
# заменим пропуски в столбце pulse через медиану по группировке surgery
print(df.groupby('surgery')['pulse'].median())
df_new['pulse'] = df['pulse'].fillna(df.groupby('surgery')['pulse'].transform('median'))
df_new.info()

surgery
1.0    70.0
2.0    56.0
Name: pulse, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     300 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          300 non-null    float64
 3   pulse                       300 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  300 non-null    float64
 6   pain                        300 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [ ]:
# Как видно из сравнения статистки после замены пропусков данные изменились несильно.
print(df['pulse'].describe())
print('-----------------------------------')
print(df_new['pulse'].describe())

count    276.000000
mean      71.913043
std       28.630557
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64
-----------------------------------
count    300.000000
mean      71.386667
std       27.579083
min       30.000000
25%       48.000000
50%       64.500000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64


In [ ]:
# заменим пропуски в столбце respiratory rate через медиану по группировке surgery
print(df.groupby('surgery')['respiratory rate'].median())
df_new['respiratory rate'] = df['respiratory rate'].fillna(df.groupby('surgery')['respiratory rate'].transform('median'))
df_new.info()

surgery
1.0    28.0
2.0    24.0
Name: respiratory rate, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     300 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          300 non-null    float64
 3   pulse                       300 non-null    float64
 4   respiratory rate            300 non-null    float64
 5   temperature of extremities  300 non-null    float64
 6   pain                        300 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [ ]:
# В сравнение видно, что уменьшилось стандартное отклонение, т.е. размах стал меньше, уменьшилось среднее, а также распределение по квартилям стало выше.
print(df['respiratory rate'].describe())
print('-----------------------------------')
print(df_new['respiratory rate'].describe())

count    242.000000
mean      30.417355
std       17.642231
min        8.000000
25%       18.500000
50%       24.500000
75%       36.000000
max       96.000000
Name: respiratory rate, dtype: float64
-----------------------------------
count    300.000000
mean      29.656667
std       15.938179
min        8.000000
25%       20.000000
50%       28.000000
75%       34.250000
max       96.000000
Name: respiratory rate, dtype: float64
